<a href="https://colab.research.google.com/github/devkumar24/NMT/blob/master/TweetEval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install transformers

     |████████████████████████████████| 2.1MB 13.1MB/s 
     |████████████████████████████████| 3.3MB 51.9MB/s 
     |████████████████████████████████| 901kB 48.4MB/s 


In [5]:
import os
import numpy as np
import pandas as pd
import torch
import transformers
import json

from tqdm import tqdm
from pathlib import Path 
from torch.utils.data import DataLoader,Dataset
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
emoji_data = "/content/drive/MyDrive/datasets/emoji"
emotion_data = "/content/drive/MyDrive/datasets/emotion"
hate_data = "/content/drive/MyDrive/datasets/hate"

In [9]:
def create_data(path,train_data = True,val_data = False,test_data = False):
    if train_data:
        file = open("{}/train_text.txt".format(path),'r')
        data_text = []
        for line in file.readlines():
            line = line[:-1]
            data_text.append(line)
        file.close()

        file = open("{}/train_labels.txt".format(path),"r")
        data_labels = []
        for line in file.readlines():
            line = line[:-1]
            data_labels.append(int(line))
        file.close()
        
        data = pd.DataFrame([data_text,data_labels],index=['Phrases','Label']).T
    
        return data
    if test_data:
        file = open("{}/test_labels.txt".format(path),"r")
        data_labels = []
        for line in file.readlines():
            line = line[:-1]
            data_labels.append(int(line))
        file.close()

        file = open("{}/test_text.txt".format(path),'r')
        data_text = []
        for line in file.readlines():
            line = line[:-1]
            data_text.append(line)
        file.close()
        
        data = pd.DataFrame([data_text,data_labels],index=['Phrases','Label']).T
    
        return data
    if val_data:
        file = open("{}/val_text.txt".format(path),'r')
        data_text = []
        for line in file.readlines():
            line = line[:-1]
            data_text.append(line)
        file.close()

        file = open("{}/val_labels.txt".format(path),'r')
        data_labels = []
        for line in file.readlines():
            line = line[:-1]
            data_labels.append(int(line))
        file.close()
        data = pd.DataFrame([data_text,data_labels],index=['Phrases','Label']).T
    
        return data
    

In [10]:
emoji_files = os.listdir(emoji_data)
emotion_files = os.listdir(emotion_data)
hate_files = os.listdir(hate_data)

In [11]:
train_emoji = create_data(path=emoji_data)
val_emoji = create_data(path = emoji_data,val_data=True,train_data = False)
test_emoji = create_data(path=emoji_data,train_data=False,val_data=False,test_data=True)

train_emotion = create_data(path=emotion_data)
val_emotion = create_data(path = emotion_data,val_data=True,train_data = False)
test_emotion = create_data(path=emotion_data,train_data=False,val_data=False,test_data=True)

train_hate = create_data(path=hate_data)
val_hate = create_data(path = hate_data,val_data=True,train_data = False)
test_hate = create_data(path=hate_data,train_data=False,val_data=False,test_data=True)

In [12]:
MAX_LEN = 256
TRAIN_BATCH_SIZE = 1
EPOCHS = 1
LEARNING_RATE = 1e-05
VAL_BATCH_SIZE = 4
TEST_BATCH_SIZE = 8

In [13]:
tokenizer = transformers.RobertaTokenizer.from_pretrained('roberta-base',truncation = True,do_lower_case = True)

In [14]:
class Data(Dataset):
    def __init__(self,dataframe,tokenize):
        self.data = dataframe
        self.tokenize = tokenize
        self.phrase = self.data.Phrases
        self.target = self.data.Label
        
    def __len__(self):
        return len(self.phrase)
        
    def __getitem__(self,index):
        text = str(self.phrase[index])
        text = " ".join(text.split())
        
        inputs = self.tokenize.encode_plus(text,None,
            add_special_tokens=True,
            max_length=MAX_LEN,
            pad_to_max_length=True,
            return_token_type_ids=True)
        
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.target[index], dtype=torch.float)
        }

In [15]:
train_data_emoji = Data(train_emoji,tokenizer)
val_data_emoji = Data(val_emoji,tokenizer)
test_data_emoji = Data(test_emoji,tokenizer)

train_data_emotion = Data(train_emotion,tokenizer)
val_data_emotion = Data(val_emotion,tokenizer)
test_data_emotion = Data(test_emotion,tokenizer)

train_data_hate = Data(train_hate,tokenizer)
val_data_hate = Data(val_hate,tokenizer)
test_data_hate = Data(test_hate,tokenizer)

In [16]:
# Recemmonded To train the data the batch size = 1 , shuffle = True
# and for validation and test put batch size = 1 and shuffle = False, because later on it will be difficult to predict the result
# I have trained with shuffle = True.
# And I can't predict the F1 Score for the test data. So, Recemmonded that to do shuffle = False while working with Val and test data
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }
val_params = {'batch_size': VAL_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }
test_params = {'batch_size': TEST_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

In [17]:
training_emoji_loader = DataLoader(train_data_emoji, **train_params)
val_emoji_loader = DataLoader(val_data_emoji,**val_params)
test_emoji_loader = DataLoader(test_data_emoji,**test_params)

training_emotion_loader = DataLoader(train_data_emotion, **train_params)
val_emotion_loader = DataLoader(val_data_emotion,**val_params)
test_emotion_loader = DataLoader(test_data_emotion,**test_params)

training_hate_loader = DataLoader(train_data_hate, **train_params)
val_hate_loader = DataLoader(val_data_hate,**val_params)
test_hate_loader = DataLoader(test_data_hate,**test_params)

In [18]:
class RoBERTa(torch.nn.Module):
    def __init__(self,no_labels):
        super(RoBERTa,self).__init__()
        self.l1 = transformers.RobertaModel.from_pretrained('roberta-base')
        self.pre_classifier = torch.nn.Linear(768,768)
        self.Dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768,no_labels)
        
    def forward(self,input_ids,attention_mask,token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.Dropout(pooler)
        output = self.classifier(pooler)
        return output

In [19]:
emoji_model = RoBERTa(no_labels=20)
emotion_model = RoBERTa(no_labels=4)
hate_model = RoBERTa(no_labels=2)

In [20]:
emoji_model.to(device)
emotion_model.to(device)
hate_model.to(device)

RoBERTa(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [21]:
loss_function = torch.nn.CrossEntropyLoss()

In [30]:
def optimizer_(model):
  return torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [22]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [31]:
def train(model,epoch,training_loader):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer = optimizer_(model)
        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer = optimizer_(model)
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [28]:
EPOCHS = 1
for epoch in range(EPOCHS):
    train(emoji_model,epoch,training_emoji_loader)

0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
3it [00:00,  4.76it/s]

Training Loss per 5000 steps: 3.059962749481201
Training Accuracy per 5000 steps: 0.0


5003it [05:02, 16.40it/s]

Training Loss per 5000 steps: 2.3479078560635536
Training Accuracy per 5000 steps: 30.313937212557487


10003it [10:04, 16.49it/s]

...

Training Loss per 5000 steps: 2.059252852830543
Training Accuracy per 5000 steps: 38.09319733721894


40003it [40:12, 16.63it/s]

Training Loss per 5000 steps: 2.0413974332584592
Training Accuracy per 5000 steps: 38.61153471163221


45000it [45:12, 16.59it/s]

The Total Accuracy for Epoch 0: 39.03111111111111
Training Loss Epoch: 2.0252654117336806
Training Accuracy Epoch: 39.03111111111111


In [32]:
EPOCHS = 1
for epoch in range(EPOCHS):
    train(emotion_model,epoch,training_emotion_loader)


0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,

2it [00:00, 15.41it/s]

Training Loss per 5000 steps: 1.318589210510254
Training Accuracy per 5000 steps: 100.0



4it [00:00, 15.20it/s]
6it [00:00, 15.36it/s]
8it [00:00, 15.41it/s]
10it [00:00, 15.42it/s]
12it [00:00, 15.42it/s]
14it [00:00, 15.53it/s]
16it [00:01, 15.47it/s]
18it [00:01, 15.55it/s]
20it [00:01, 15.58it/s]
22it [00:01, 15.54it/s]
24it [00:01, 15.58it/s]
26it [00:01, 15.37it/s]
28it [00:01, 15.46it/s]
30it [00:01, 15.53it/s]
32it [00:02, 15.61it/s]
34it [00:02, 15.64it/s]
36it [00:02, 15.53it/s]
38it [00:02, 15.58it/s]
40it [00:02, 15.53it/s]
42it [00:02, 15.59it/s]
44it [00:02, 15.63it/s]
46it [00:02, 15.55it/s]
48it [00:03, 15.56it/s]
50it [00:03, 15.55it/s]
52it [00:03, 15.58it/s]
54it [00:03, 15.63it/s]
56it [00:03, 15.64it/s]
58it [00:03, 15.69it/s]
60it [00:03, 15.71it/s]
62it [00:03, 15.71it/s]
64it [00:04, 15.70it/s]
66it [00:04, 15.70it/s]
68it [00:04, 15.46it/s]
70it [00:04, 15.53it/s]
72it [00:04, 15.59it/s]
74it [00:04, 15.44it/s]
76it [00:04, 15.52it/s]
78it [00:05, 15.54it/s]
80it [00:05, 15.62it/s]
82it [00:05, 15.63it/s]
84it [00:05, 15.65it/s]
86it [00:05, 15.60

The Total Accuracy for Epoch 0: 65.48971446116057
Training Loss Epoch: 2.061476450990593
Training Accuracy Epoch: 65.48971446116057


In [33]:
EPOCHS = 1
for epoch in range(EPOCHS):
    train(hate_model,epoch,training_hate_loader)


0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,

2it [00:00, 15.66it/s]

Training Loss per 5000 steps: 0.6156365871429443
Training Accuracy per 5000 steps: 100.0



4it [00:00, 15.30it/s]
6it [00:00, 15.41it/s]
8it [00:00, 15.33it/s]
10it [00:00, 15.44it/s]
12it [00:00, 15.43it/s]
14it [00:00, 15.47it/s]
16it [00:01, 15.48it/s]
18it [00:01, 15.58it/s]
20it [00:01, 15.64it/s]
22it [00:01, 15.55it/s]
24it [00:01, 15.53it/s]
26it [00:01, 15.52it/s]
28it [00:01, 15.57it/s]
30it [00:01, 15.59it/s]
32it [00:02, 15.61it/s]
34it [00:02, 15.62it/s]
36it [00:02, 15.66it/s]
38it [00:02, 15.48it/s]
40it [00:02, 15.49it/s]
42it [00:02, 15.39it/s]
44it [00:02, 15.49it/s]
46it [00:02, 15.55it/s]
48it [00:03, 15.57it/s]
50it [00:03, 15.57it/s]
52it [00:03, 15.58it/s]
54it [00:03, 15.62it/s]
56it [00:03, 15.65it/s]
58it [00:03, 15.65it/s]
60it [00:03, 15.64it/s]
62it [00:03, 15.69it/s]
64it [00:04, 15.63it/s]
66it [00:04, 15.63it/s]
68it [00:04, 15.65it/s]
70it [00:04, 15.68it/s]
72it [00:04, 15.66it/s]
74it [00:04, 15.67it/s]
76it [00:04, 15.67it/s]
78it [00:05, 15.67it/s]
80it [00:05, 15.64it/s]
82it [00:05, 15.66it/s]
84it [00:05, 15.67it/s]
86it [00:05, 15.68

Training Loss per 5000 steps: 2.505201916757921
Training Accuracy per 5000 steps: 70.34593081383723



5004it [05:28, 14.96it/s]
5006it [05:28, 15.10it/s]
5008it [05:28, 15.23it/s]
5010it [05:28, 15.17it/s]
5012it [05:28, 15.29it/s]
5014it [05:28, 15.11it/s]
5016it [05:29, 15.21it/s]
5018it [05:29, 15.29it/s]
5020it [05:29, 15.35it/s]
5022it [05:29, 15.41it/s]
5024it [05:29, 15.11it/s]
5026it [05:29, 15.06it/s]
5028it [05:29, 15.13it/s]
5030it [05:29, 15.22it/s]
5032it [05:30, 15.32it/s]
5034it [05:30, 15.41it/s]
5036it [05:30, 15.40it/s]
5038it [05:30, 15.40it/s]
5040it [05:30, 15.45it/s]
5042it [05:30, 15.46it/s]
5044it [05:30, 15.42it/s]
5046it [05:30, 15.44it/s]
5048it [05:31, 15.46it/s]
5050it [05:31, 15.51it/s]
5052it [05:31, 15.41it/s]
5054it [05:31, 15.46it/s]
5056it [05:31, 15.51it/s]
5058it [05:31, 15.53it/s]
5060it [05:31, 15.46it/s]
5062it [05:32, 15.44it/s]
5064it [05:32, 15.44it/s]
5066it [05:32, 15.50it/s]
5068it [05:32, 15.53it/s]
5070it [05:32, 15.45it/s]
5072it [05:32, 15.43it/s]
5074it [05:32, 15.37it/s]
5076it [05:32, 15.33it/s]
5078it [05:33, 15.33it/s]
5080it [05:

The Total Accuracy for Epoch 0: 73.45555555555555
Training Loss Epoch: 2.754442790189051
Training Accuracy Epoch: 73.45555555555555


In [34]:
output_model_file = 'TweetEval-emoji.bin'
output_vocab_file = './'

model_to_save = emoji_model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)



output_model_file = 'TweetEval-emotion.bin'
output_vocab_file = './'

model_to_save = emoji_model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)



output_model_file = 'TweetEval-hate.bin'
output_vocab_file = './'

model_to_save = emoji_model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

('./vocab.json', './merges.txt')

In [35]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accuracy(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    return epoch_accu


In [ ]:
emoji_model = torch.load('./emotion/TweetEval-emotion.bin',map_location=torch.device('cpu'))
emotion_model = torch.load('./emotion/TweetEval-emotion.bin',map_location=torch.device('cpu'))
hate_model = torch.load('./hate/TweetEval-hate.bin',map_location=torch.device('cpu'))

In [36]:
acc = valid(emoji_model, val_emoji_loader)
print("Accuracy on test data = %0.2f%%" % acc)

acc = valid(emotion_model, val_emotion_loader)
print("Accuracy on test data = %0.2f%%" % acc)

acc = valid(hate_model, val_hate_loader)
print("Accuracy on val data = %0.2f%%" % acc)


0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,

3it [00:00, 23.26it/s]

Validation Loss per 100 steps: 2.260939121246338
Validation Accuracy per 100 steps: 25.0



6it [00:00, 23.53it/s]
9it [00:00, 23.80it/s]
12it [00:00, 23.96it/s]
15it [00:00, 24.05it/s]
18it [00:00, 24.17it/s]
21it [00:00, 24.25it/s]
24it [00:00, 24.38it/s]
27it [00:01, 24.47it/s]
30it [00:01, 24.35it/s]
33it [00:01, 24.22it/s]
36it [00:01, 24.34it/s]
39it [00:01, 24.50it/s]
42it [00:01, 24.43it/s]
45it [00:01, 24.49it/s]
48it [00:01, 24.42it/s]
51it [00:02, 24.49it/s]
54it [00:02, 24.55it/s]
57it [00:02, 24.65it/s]
60it [00:02, 24.63it/s]
63it [00:02, 24.77it/s]
66it [00:02, 24.81it/s]
69it [00:02, 24.87it/s]
72it [00:02, 24.85it/s]
75it [00:03, 24.74it/s]
78it [00:03, 24.79it/s]
81it [00:03, 24.81it/s]
84it [00:03, 24.81it/s]
87it [00:03, 24.87it/s]
90it [00:03, 24.91it/s]
93it [00:03, 24.83it/s]
96it [00:03, 24.83it/s]
99it [00:04, 24.86it/s]
102it [00:04, 24.90it/s]
105it [00:04, 24.88it/s]
108it [00:04, 24.88it/s]
111it [00:04, 24.78it/s]
114it [00:04, 24.60it/s]
117it [00:04, 24.49it/s]
120it [00:04, 24.48it/s]
123it [00:05, 24.45it/s]
126it [00:05, 24.33it/s]
129it [0

Validation Loss Epoch: 2.3835136318206787
Validation Accuracy Epoch: 26.66
Accuracy on test data = 26.66%
Validation Loss per 100 steps: 6.440564155578613
Validation Accuracy per 100 steps: 50.0



6it [00:00, 25.00it/s]
9it [00:00, 24.91it/s]
12it [00:00, 24.92it/s]
15it [00:00, 24.89it/s]
18it [00:00, 24.91it/s]
21it [00:00, 24.84it/s]
24it [00:00, 24.84it/s]
27it [00:01, 24.83it/s]
30it [00:01, 24.81it/s]
33it [00:01, 24.77it/s]
36it [00:01, 24.80it/s]
39it [00:01, 24.86it/s]
42it [00:01, 24.73it/s]
45it [00:01, 24.68it/s]
48it [00:01, 24.76it/s]
51it [00:02, 24.80it/s]
54it [00:02, 24.80it/s]
57it [00:02, 24.86it/s]
60it [00:02, 24.68it/s]
63it [00:02, 24.65it/s]
66it [00:02, 24.58it/s]
69it [00:02, 24.46it/s]
72it [00:02, 24.41it/s]
75it [00:03, 24.43it/s]
78it [00:03, 24.47it/s]
81it [00:03, 24.40it/s]
84it [00:03, 24.21it/s]
87it [00:03, 24.27it/s]
90it [00:03, 24.28it/s]
94it [00:03, 24.67it/s]

0it [00:00, ?it/s]
3it [00:00, 25.49it/s]

Validation Loss Epoch: 2.3763593900266313
Validation Accuracy Epoch: 76.47058823529412
Accuracy on test data = 76.47%
Validation Loss per 100 steps: 3.624500036239624
Validation Accuracy per 100 steps: 75.0



6it [00:00, 25.09it/s]
9it [00:00, 24.82it/s]
12it [00:00, 24.60it/s]
15it [00:00, 24.44it/s]
18it [00:00, 24.38it/s]
21it [00:00, 24.37it/s]
24it [00:00, 24.46it/s]
27it [00:01, 24.54it/s]
30it [00:01, 24.58it/s]
33it [00:01, 24.63it/s]
36it [00:01, 24.68it/s]
39it [00:01, 24.67it/s]
42it [00:01, 24.72it/s]
45it [00:01, 24.76it/s]
48it [00:01, 24.73it/s]
51it [00:02, 24.74it/s]
54it [00:02, 24.77it/s]
57it [00:02, 24.81it/s]
60it [00:02, 24.83it/s]
63it [00:02, 24.84it/s]
66it [00:02, 24.89it/s]
69it [00:02, 24.86it/s]
72it [00:02, 24.92it/s]
75it [00:03, 24.89it/s]
78it [00:03, 24.82it/s]
81it [00:03, 24.80it/s]
84it [00:03, 24.79it/s]
87it [00:03, 24.80it/s]
90it [00:03, 24.85it/s]
93it [00:03, 24.87it/s]
96it [00:03, 24.73it/s]
99it [00:04, 24.67it/s]
102it [00:04, 24.66it/s]
105it [00:04, 24.60it/s]
108it [00:04, 24.50it/s]
111it [00:04, 24.41it/s]
114it [00:04, 24.28it/s]
117it [00:04, 24.18it/s]
120it [00:04, 24.13it/s]
123it [00:04, 24.30it/s]
126it [00:05, 24.35it/s]
129it [0

Validation Loss Epoch: 3.6996270042584403
Validation Accuracy Epoch: 73.2
Accuracy on val data = 73.20%


In [37]:
acc = valid(emoji_model, test_emoji_loader)
print("Accuracy on test data = %0.2f%%" % acc)

acc = valid(emotion_model, test_emotion_loader)
print("Accuracy on test data = %0.2f%%" % acc)

acc = valid(hate_model, test_hate_loader)
print("Accuracy on val data = %0.2f%%" % acc)


0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,

2it [00:00, 11.68it/s]

Validation Loss per 100 steps: 0.9777043461799622
Validation Accuracy per 100 steps: 62.5



4it [00:00, 12.04it/s]
6it [00:00, 12.32it/s]
8it [00:00, 12.61it/s]
10it [00:00, 12.79it/s]
12it [00:00, 12.98it/s]
14it [00:01, 13.08it/s]
16it [00:01, 13.18it/s]
18it [00:01, 13.23it/s]
20it [00:01, 13.07it/s]
22it [00:01, 13.21it/s]
24it [00:01, 13.29it/s]
26it [00:01, 13.31it/s]
28it [00:02, 13.35it/s]
30it [00:02, 13.34it/s]
32it [00:02, 13.39it/s]
34it [00:02, 13.42it/s]
36it [00:02, 13.41it/s]
38it [00:02, 13.41it/s]
40it [00:03, 13.41it/s]
42it [00:03, 13.44it/s]
44it [00:03, 13.48it/s]
46it [00:03, 13.48it/s]
48it [00:03, 13.47it/s]
50it [00:03, 13.46it/s]
52it [00:03, 13.50it/s]
54it [00:04, 13.44it/s]
56it [00:04, 13.41it/s]
58it [00:04, 13.37it/s]
60it [00:04, 13.37it/s]
62it [00:04, 13.35it/s]
64it [00:04, 13.33it/s]
66it [00:04, 13.33it/s]
68it [00:05, 13.32it/s]
70it [00:05, 13.32it/s]
72it [00:05, 13.32it/s]
74it [00:05, 13.24it/s]
76it [00:05, 13.26it/s]
78it [00:05, 13.26it/s]
80it [00:06, 13.25it/s]
82it [00:06, 13.29it/s]
84it [00:06, 13.33it/s]
86it [00:06, 13.31

Validation Loss per 100 steps: 1.7916486149333855
Validation Accuracy per 100 steps: 45.38842231553689



5004it [06:16, 13.10it/s]
5006it [06:16, 13.13it/s]
5008it [06:16, 13.10it/s]
5010it [06:16, 13.18it/s]
5012it [06:17, 13.17it/s]
5014it [06:17, 13.05it/s]
5016it [06:17, 13.08it/s]
5018it [06:17, 13.11it/s]
5020it [06:17, 13.14it/s]
5022it [06:17, 13.15it/s]
5024it [06:17, 13.13it/s]
5026it [06:18, 13.10it/s]
5028it [06:18, 13.08it/s]
5030it [06:18, 13.08it/s]
5032it [06:18, 13.22it/s]
5034it [06:18, 13.33it/s]
5036it [06:18, 13.40it/s]
5038it [06:19, 13.41it/s]
5040it [06:19, 13.28it/s]
5042it [06:19, 13.31it/s]
5044it [06:19, 13.40it/s]
5046it [06:19, 13.46it/s]
5048it [06:19, 13.49it/s]
5050it [06:19, 13.38it/s]
5052it [06:20, 13.41it/s]
5054it [06:20, 13.43it/s]
5056it [06:20, 13.48it/s]
5058it [06:20, 13.49it/s]
5060it [06:20, 13.30it/s]
5062it [06:20, 13.30it/s]
5064it [06:20, 13.21it/s]
5066it [06:21, 13.20it/s]
5068it [06:21, 13.17it/s]
5070it [06:21, 13.15it/s]
5072it [06:21, 13.20it/s]
5074it [06:21, 13.29it/s]
5076it [06:21, 13.39it/s]
5078it [06:22, 13.29it/s]
5080it [06:

Validation Loss Epoch: 1.7955281915950776
Validation Accuracy Epoch: 45.224
Accuracy on test data = 45.22%
Validation Loss per 100 steps: 3.3212766647338867
Validation Accuracy per 100 steps: 75.0



4it [00:00, 13.49it/s]
6it [00:00, 13.34it/s]
8it [00:00, 13.39it/s]
10it [00:00, 13.42it/s]
12it [00:00, 13.39it/s]
14it [00:01, 13.43it/s]
16it [00:01, 13.43it/s]
18it [00:01, 13.45it/s]
20it [00:01, 13.42it/s]
22it [00:01, 13.45it/s]
24it [00:01, 13.49it/s]
26it [00:01, 13.26it/s]
28it [00:02, 13.35it/s]
30it [00:02, 13.40it/s]
32it [00:02, 13.41it/s]
34it [00:02, 13.47it/s]
36it [00:02, 13.51it/s]
38it [00:02, 13.49it/s]
40it [00:02, 13.36it/s]
42it [00:03, 13.27it/s]
44it [00:03, 13.20it/s]
46it [00:03, 13.06it/s]
48it [00:03, 13.05it/s]
50it [00:03, 13.14it/s]
52it [00:03, 13.22it/s]
54it [00:04, 13.32it/s]
56it [00:04, 13.38it/s]
58it [00:04, 13.43it/s]
60it [00:04, 13.46it/s]
62it [00:04, 13.46it/s]
64it [00:04, 13.50it/s]
66it [00:04, 13.47it/s]
68it [00:05, 13.35it/s]
70it [00:05, 13.23it/s]
72it [00:05, 13.13it/s]
74it [00:05, 13.20it/s]
76it [00:05, 13.07it/s]
78it [00:05, 13.06it/s]
80it [00:06, 13.04it/s]
82it [00:06, 13.04it/s]
84it [00:06, 12.96it/s]
86it [00:06, 12.94

Validation Loss Epoch: 2.2508304662887078
Validation Accuracy Epoch: 78.60661505981703
Accuracy on test data = 78.61%
Validation Loss per 100 steps: 9.224122047424316
Validation Accuracy per 100 steps: 50.0



4it [00:00, 13.40it/s]
6it [00:00, 13.11it/s]
8it [00:00, 13.09it/s]
10it [00:00, 13.11it/s]
12it [00:00, 13.17it/s]
14it [00:01, 13.26it/s]
16it [00:01, 13.28it/s]
18it [00:01, 13.22it/s]
20it [00:01, 13.12it/s]
22it [00:01, 13.06it/s]
24it [00:01, 12.95it/s]
26it [00:01, 12.95it/s]
28it [00:02, 12.94it/s]
30it [00:02, 12.87it/s]
32it [00:02, 12.83it/s]
34it [00:02, 12.74it/s]
36it [00:02, 12.73it/s]
38it [00:02, 12.84it/s]
40it [00:03, 12.83it/s]
42it [00:03, 13.02it/s]
44it [00:03, 13.02it/s]
46it [00:03, 13.01it/s]
48it [00:03, 12.96it/s]
50it [00:03, 13.09it/s]
52it [00:03, 13.14it/s]
54it [00:04, 13.22it/s]
56it [00:04, 13.26it/s]
58it [00:04, 13.31it/s]
60it [00:04, 13.31it/s]
62it [00:04, 13.38it/s]
64it [00:04, 13.41it/s]
66it [00:05, 13.23it/s]
68it [00:05, 13.14it/s]
70it [00:05, 13.13it/s]
72it [00:05, 13.25it/s]
74it [00:05, 13.33it/s]
76it [00:05, 13.40it/s]
78it [00:05, 13.43it/s]
80it [00:06, 13.44it/s]
82it [00:06, 13.41it/s]
84it [00:06, 13.49it/s]
86it [00:06, 13.48

Validation Loss Epoch: 9.040437224571404
Validation Accuracy Epoch: 48.148148148148145
Accuracy on val data = 48.15%


In [40]:
def predict(model, test_loader):
    model.eval()
    output = []
    with torch.no_grad():
        for _, data in tqdm(enumerate(test_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids).squeeze()
            output.append(outputs)
    return output

In [41]:
emoji_pred = predict(emoji_model,test_emoji_loader)
emotion_pred = predict(emotion_model,test_emotion_loader)
hate_pred = predict(hate_model,test_hate_loader)


0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,

3it [00:00, 17.69it/s]
5it [00:00, 16.66it/s]
7it [00:00, 15.99it/s]
9it [00:00, 15.60it/s]
11it [00:00, 15.34it/s]
13it [00:00, 15.19it/s]
15it [00:00, 15.10it/s]
17it [00:01, 15.02it/s]
19it [00:01, 14.96it/s]
21it [00:01, 14.91it/s]
23it [00:01, 14.89it/s]
25it [00:01, 14.85it/s]
27it [00:01, 14.83it/s]
29it [00:01, 14.81it/s]
31it [00:02, 14.81it/s]
33it [00:02, 14.87it/s]
35it [00:02, 14.85it/s]
37it [00:02, 14.86it/s]
39it [00:02, 14.87it/s]
4

In [74]:
emoji_predictions = list()
for i in range(len(emoji_pred)):
  for j in range(TEST_BATCH_SIZE):
    out_arr = emoji_pred[i][j]
    out_arr = out_arr.cpu().data.numpy()
    max_id = np.argmax(out_arr)
    emoji_predictions.append(max_id)

In [75]:
len(emoji_predictions)

50000

In [78]:
file = open('emoji_pred.txt','w')
for i in emoji_predictions:
  file.write(str(i))
file.close()

In [83]:
emotion_predictions = list()
for i in range(len(emotion_pred)):
  for j in range(len(emotion_pred[i])):
    out_arr = emotion_pred[i][j]
    out_arr = out_arr.cpu().data.numpy()
    max_id = np.argmax(out_arr)
    emotion_predictions.append(max_id)

In [81]:
len(emotion_predictions)

1421

In [82]:
file = open('emotion_pred.txt','w')
for i in emotion_predictions:
  file.write(str(i))
file.close()

In [84]:
hate_predictions = list()
for i in range(len(hate_pred)):
  for j in range(len(hate_pred[i])):
    out_arr = hate_pred[i][j]
    out_arr = out_arr.cpu().data.numpy()
    max_id = np.argmax(out_arr)
    hate_predictions.append(max_id)

In [85]:
len(hate_predictions)

2970

In [86]:
file = open('hate_pred.txt','w')
for i in hate_predictions:
  file.write(str(i))
file.close()